In [ ]:
# Задание 1
Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
Cохранить ее в pandas dataframe
Cохранить pandas dataframe в excel
Cохранить pandas dataframe в pickle
Cохранить pandas dataframe в БД

In [1]:
import requests
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [2]:
# cобираем айди (циклом по 100)
unik=[]
limit_ = 100
for i in tqdm(range (1,10700,100)):
    offset_ = i
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    objects_data = res.json()
    objects_list = objects_data.get('data').get('list')
    objids = [x.get('objId') for x in objects_list]
    unik.extend(objids)
    #print(i)

In [12]:
#нормализуем и записывем инфо
dfs = pd.DataFrame()
for ids in tqdm(unik):
    url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/' + str(ids)
    res = requests.get(url)
    dfs=dfs.append(pd.json_normalize(res.json().get('data')))
    #print(unik.index(ids))
dfs.to_excel('stop_loc.xlsx')#записываем в иксель
dfs.to_pickle("./dummy.pkl")#записываем в пикл

In [14]:
#записываем в БД
import sqlite3 as sl
con = sl.connect('my-test.db')
dfs1 = dfs.applymap(str)
dfs1.to_sql('dfs2', con)

In [3]:
dfs1 = pd.DataFrame()
for ids in tqdm(unik):
    url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/' + str(ids)
    res = requests.get(url)
    dfs1=dfs1.append(pd.json_normalize(res.json().get('data').get('photoRenderDTO')))

In [31]:
import os

In [ ]:
#цикл для сохранения картинок в папке с айди стройки
for i, row in tqdm(dfs1[['objId','objRenderPhotoUrl']].iterrows()):
    if not os.path.exists(str(row['objId'])):
        os.makedirs(str(row['objId']))
        urlimg=row['objRenderPhotoUrl']
        res2=requests.get(urlimg)
        imageFile=open(os.path.join(str(row['objId']),str(i)),'wb')
        for chunk in res2.iter_content(100000):
            imageFile.write(chunk)
        imageFile.close()
        #imageFile.write(row['objRenderPhotoUrl'].content)
        #imageFile.close()
    else:
        urlimg=row['objRenderPhotoUrl']
        res2=requests.get(urlimg)
        imageFile=open(os.path.join(str(row['objId']),str(i)),'wb')
        for chunk in res2.iter_content(100000):
            imageFile.write(chunk)
        imageFile.close()